In [8]:
import numpy as np
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sc
import time
import sys
%cd /Users/James/Desktop/Eigth Semester/Stochastic Modelling/NFL_Project/Week Dependent Analysis

/Users/James/Desktop/Eigth Semester/Stochastic Modelling/NFL_Project/Week Dependent Analysis


In [9]:
data = pd.read_csv('week_data_2018.csv',index_col=0)
data = data.drop(['game_id', 'home_team'], axis=1)
data_plt = np.array(data)
print(data.head())
print(np.shape(data))

   winning_team  losing_team  diff_abs  week
1             1           18       3.5     1
2             2           24      41.5     1
3            22            9       2.5     1
4            17            8      12.5     1
5             7           20       4.5     1
(256, 4)


In [10]:
# filtering out games after a certain week (here, all weeks less than 10)
data["week"] = data[data["week"] < 16]["week"]
data = data.dropna()
data = data.drop("week", axis = 1)
print(np.shape(data))

(224, 3)


In [11]:
data_np = np.array(data)

## Creating the Pij Matrix

In [12]:
Pij = np.zeros((32,32))

In [13]:
for iCnt in range(np.shape(data_np)[0]):
    winner = int(data_np[iCnt,0])-1
    loser =  int(data_np[iCnt,1])-1
    # checking if an interdivisional game has occurred
    if (Pij[loser,winner] == 0) & (Pij[winner,loser] == 0):
        Pij[winner,loser] = data_np[iCnt,2]
    # when the other team had won the first game
    elif (Pij[loser,winner] > 0):
        if (Pij[loser,winner] > data_np[iCnt,2]):
            Pij[loser,winner] = (Pij[loser,winner] - data_np[iCnt,2])/2
        else: 
            Pij[winner,loser] = -(Pij[loser,winner] - data_np[iCnt,2])/2
            Pij[loser,winner] = 0
    # when the same team won the first game
    else: 
        Pij[winner,loser] = (Pij[winner,loser] + data_np[iCnt,2])/2

# Making the Function

In [14]:
def week_UB_Pij(data, weekub, method, weighted = True):
    
    # Getting the Data in the right form
    data = data.drop(['game_id', 'home_team'], axis=1)
    if method == "less":
        # IF METHOD IS WEEKS LESS THAN:
        data["week"] = data[data["week"] < weekub]["week"]
    if method == "equal":
        # If METHOD IS GETTING THE EXACT WEEK
        data["week"] = data[data["week"] == weekub]["week"]
    data = data.dropna()
    data = data.drop("week", axis = 1)
    data_np = np.array(data)
    
    Pij = np.zeros((32,32))
    
    # Computing the Pij Matrix
    for iCnt in range(np.shape(data_np)[0]):
        winner = int(data_np[iCnt,0])-1
        loser =  int(data_np[iCnt,1])-1
        # checking if an interdivisional game has occurred
        if (Pij[loser,winner] == 0) & (Pij[winner,loser] == 0):
            Pij[winner,loser] = data_np[iCnt,2]
        # when the other team had won the first game
        elif (Pij[loser,winner] > 0):
            if (Pij[loser,winner] > data_np[iCnt,2]):
                Pij[loser,winner] = (Pij[loser,winner] - data_np[iCnt,2])/2
            else: 
                Pij[winner,loser] = -(Pij[loser,winner] - data_np[iCnt,2])/2
                Pij[loser,winner] = 0
        # when the same team won the first game
        else: 
            Pij[winner,loser] = (Pij[winner,loser] + data_np[iCnt,2])/2
            
    if(weighted == False):
                Pij[Pij!=0] = 1
            
    # making sure none of the columns sum to 0
    if len(np.where(Pij.sum(axis = 0) == 0)[0]) > 0:
        empty_vect = np.where(Pij.sum(axis = 0) == 0)[0]
        for iCnt in range(len(empty_vect)):
            Pij[empty_vect[iCnt],empty_vect[iCnt]] = 1
    
    # making the Pij matrix
    Pij = Pij/Pij.sum(axis=0)
        
    # returning the Pij Matrix
    return Pij

In [15]:
data = pd.read_csv('week_data_2018.csv',index_col=0)
Pij = week_UB_Pij(data, 16, method = "less")

# Determining Scores (FOR DESCRIPTIVE PURPOSES ONLY)

In [16]:
num_weeks = 17
scores_mat = np.zeros((32,num_weeks))
for iCnt in range(num_weeks):
    Pij_new = week_UB_Pij(data, iCnt + 2, method = "less")
    
    # making the teleporting probabilities uniform
    v = np.ones(32)/32
    v = v[np.newaxis]
    v = v.T
    # Setting eT
    eT = np.ones(32)[np.newaxis]
    # initializing our first guess
    x0 = np.ones(32)/32
    x0 = x0[np.newaxis]
    x0 = x0.T
            
    # performing the simulation
    for i in range(5000):
        x0 = .5*np.dot(Pij_new,x0) + (1-.5)*v
            
    x0 = np.squeeze(x0)
    
    # updating the scores matrix
    scores_mat[0:32,iCnt] = x0

# Figuring Out the Teams That were Played at What Week

In [17]:
teams = pd.read_csv('teams_2018.csv',index_col=0)
team_names = np.array(teams["team_name"])
team_opps = [''] * 17;
team = 8

print(team_names)

for iCnt in range(np.shape(data_plt)[0]):
    if data_plt[iCnt,0] == team  + 1:
        team_opps[int(data_plt[iCnt,3])-1] = team_names[int(data_plt[iCnt,1])-1]
    if data_plt[iCnt,1] == team + 1:
        team_opps[int(data_plt[iCnt,3])-1] = team_names[int(data_plt[iCnt,0])-1]
    
team_opps = np.array(team_opps)
team_opps[team_opps == ''] = "BYE"
print(team_opps)

['PHI' 'BAL' 'NYG' 'NO' 'NE' 'MIN' 'MIA' 'IND' 'CLE' 'LAC' 'DEN' 'CAR'
 'ARI' 'GB' 'DET' 'OAK' 'CIN' 'ATL' 'WAS' 'TEN' 'TB' 'PIT' 'NYJ' 'BUF' 'SF'
 'LA' 'JAX' 'DAL' 'CHI' 'HOU' 'KC' 'SEA']
['PIT' 'NO' 'NYJ' 'OAK' 'BAL' 'LAC' 'TB' 'PIT' 'KC' 'ATL' 'BYE' 'CIN' 'HOU'
 'CAR' 'DEN' 'CIN' 'BAL']


In [30]:
teams = pd.read_csv('teams_2018.csv',index_col=0)

def team_plotter(scores_mat, teams, team, display_opps = False):
    
    team_names = np.array(teams["team_name"])
    # getting the labels ---------
    team_opps = [''] * 17
    for iCnt in range(np.shape(data_plt)[0]):
        if data_plt[iCnt,0] == team  + 1:
            team_opps[int(data_plt[iCnt,3])-1] = team_names[int(data_plt[iCnt,1])-1]
        if data_plt[iCnt,1] == team + 1:
            team_opps[int(data_plt[iCnt,3])-1] = team_names[int(data_plt[iCnt,0])-1]
    
    team_opps = np.array(team_opps)
    team_opps[team_opps == ''] = "BYE"
    # ------------------------------
   

    plt.figure(figsize=(15,7))
    plt.plot(np.arange(17)+1, scores_mat[team,],'bo-')
    plt.grid()
    plt.xlabel('Week of NFL Season')
    plt.ylabel('PageRank Score')
    plt.ylim(0,0.1)
    plt.title("Scores for %s" %team_names[team])
    
    # adding the annotation ---------
    if display_opps == True:
        for label, x, y in zip(team_opps, np.arange(17)+1 , scores_mat[team,]):
            plt.annotate(
                label,
                xy=(x, y), xytext=(0, 30),
                textcoords='offset points', va='bottom',
                bbox=dict(boxstyle='round,pad=.5', fc='red', alpha=0.25),
                arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
    # --------------

    plt.show()
    
    

In [27]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [28]:
names_list = []
team_names = np.array(teams["team_name"])
for iCnt in range(32):
    names_list.append((team_names[iCnt], iCnt))
names_list.sort()

In [31]:
interact(team_plotter,scores_mat =fixed(scores_mat), teams = fixed(teams), \
         team = names_list)

A Jupyter Widget

<function __main__.team_plotter>

In [525]:
P = np.array([[0,.5,1,1],[.5,0,0,0],[.5,.5,0,0],[0,0,0,0]])

In [553]:
alpha = .5

In [554]:
# making the teleporting probabilities uniform
v = np.ones(4)/4
v = v[np.newaxis]
v = v.T
# Setting eT
eT = np.ones(4)[np.newaxis]
# initializing our first guess
x0 = np.ones(4)/4
x0 = x0[np.newaxis]
x0 = x0.T
            
# performing the simulation
for i in range(2000):
    x0 = alpha*np.dot(P,x0) + (1-alpha)*v
            
x0 = np.squeeze(x0)

In [555]:
print(x0)

[ 0.38   0.22   0.275  0.125]
